In [1]:
import os
import json
from getpass import getpass

In [2]:
os.environ['OPENAI_API_KEY'] = getpass()

In [3]:
from openai import OpenAI

client = OpenAI()

In [5]:
with open("../data/math_test_private.json", encoding="utf-8") as f:
    data = json.load(f)

In [6]:
prime = [
    ("Một xưởng may trong tuần thứ nhất thực hiện được 3/8 kế hoạch tháng, tuần thứ hai thực hiện được 3/16 kế hoạch, trong tuần thứ ba thực hiện được 1/3 kế hoạch. Để hoàn thành kế hoạch của tháng thì trong tuần cuối xưởng phải thực hiện bao nhiêu phần kế hoạch?\nA. 5/48\nB. 43/48\nC. 11/48\nD. 27/48", "A. 5/48"),
    ("Mệnh đề nào sau đây là phủ định của mệnh đề 'Mọi động vật đều di chuyển'?A. Mọi động vật đều không di chuyển\nB. Mọi động vật đều đứng yên\nC. Có ít nhất một động vật không di chuyển\nD. Có ít nhất một động vật di chuyển", "C. Có ít nhất một động vật không di chuyển"),
    ("Một bài trắc nghiệm khách quan có 10 câu hỏi. Mỗi câu có 4 phương án trả lời. Có bao nhiêu phương án trả lời?A. 40\nB. 10^4\nC. 4^10\nD. 4", "C. 4^10")
]

In [7]:
outputs = []
chat_completions = []

In [9]:
for idx, sample in enumerate(data['data']):
    message = sample["question"]
    for choice in sample["choices"]:
        message += f"\n{choice}"

    print(f"At datapoint {idx}\n")
    print(message,"\n")

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[
            {"role": "system", "content": "You are a helpful assistant that solves Vietnamese word math problem in the form of multi-choice questions, specifically you have to choose the one correct option among multiple options and your reply must contain no more information other than the chosen option itself."},
            {"role": "system", "name":"example_user", "content": prime[0][0]},
            {"role": "system", "name": "example_assistant", "content": prime[0][1]},
            {"role": "system", "name":"example_user", "content": prime[1][0]},
            {"role": "system", "name": "example_assistant", "content": prime[1][1]},
            {"role": "system", "name":"example_user", "content": prime[2][0]},
            {"role": "system", "name": "example_assistant", "content": prime[2][1]},
            {"role": "user", "content": message},
        ],
        temperature=0,
    )

    chat_completions.append(response)
    output = {}
    output["answer"] = response.choices[0].message.content
    output["id"] = sample["id"]
    print(output["answer"], "\n")
    print("=================================================\n")
    outputs.append(output)


At datapoint 0

Kết quả của 1,05 m^{3} = …….. dm^{3} ?
A. 105
B. 1005
C. 1050
D. 1500 



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
outs = []
for sample, chat in zip(data['data'], chat_completions):
    out = {}
    out['id'] = sample['id']
    out['answer'] = chat.choices[0].message.content
    outs.append(out)
outs

[{'id': '01-0203', 'answer': 'B. 45 000 000 đồng'},
 {'id': '01-0206', 'answer': 'A. 24 phút'},
 {'id': '01-0207', 'answer': 'B. 4 lần'},
 {'id': '01-0209', 'answer': 'C. 25%'},
 {'id': '01-0210', 'answer': 'B. 200m'},
 {'id': '01-0211', 'answer': 'B. 5,621'},
 {'id': '01-0214', 'answer': 'C. 40%'},
 {'id': '01-0219', 'answer': 'A. 7 giờ 17 phút'},
 {'id': '01-0221', 'answer': 'C. 0,75'},
 {'id': '01-0222', 'answer': 'C. 67,919'},
 {'id': '01-0223', 'answer': 'B. 24'},
 {'id': '01-0224', 'answer': 'D. 39870'},
 {'id': '01-0225', 'answer': 'C. \\frac{7}{100}'},
 {'id': '01-0227', 'answer': 'C. 5 chục'},
 {'id': '01-0232', 'answer': 'B. 15%'},
 {'id': '01-0234', 'answer': 'A. \\frac{1}{8}'},
 {'id': '01-0237', 'answer': 'C. 350'},
 {'id': '01-0239', 'answer': 'C. 5019'},
 {'id': '01-0240', 'answer': 'C. 0,18 giờ'},
 {'id': '01-0241', 'answer': 'A. 36,992'},
 {'id': '01-0243', 'answer': 'B. 500 000'},
 {'id': '01-0245', 'answer': 'C. 37,4'},
 {'id': '01-0246', 'answer': 'D. 10,5'},
 {'id'

In [ ]:
import csv
with open('submit_few_shot.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['id', 'answer'])
    writer.writeheader()
    writer.writerows(outs)

In [ ]:
import pickle
with open("bin_few_shot_submit.dat", "wb") as f:
    pickle.dump(chat_completions, f)